Engineer: Adeola Odunewu
Intern: FlipRobo LLC DS1123
Project: Zomato Resturant Project

In [ ]:
import joblib
import warnings

import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns

import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.svm import SVR, SVC

from scipy.stats import randint

from statsmodels.formula.api import ols
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_curve, auc

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest, f_classif, RFE

from sklearn.linear_model import LogisticRegression, LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor

warnings.filterwarnings("ignore", message=".*deprecated.*")

In [ ]:

# File paths for the CSV files
url1 = 'https://github.com/FlipRoboTechnologies/ML_-Datasets/raw/main/Z_Restaurant/Country-Code.xlsx'
url2 = 'https://raw.githubusercontent.com/FlipRoboTechnologies/ML_-Datasets/main/Z_Restaurant/zomato.csv'

# Read CSV files into DataFrame objects
df1 = pd.read_excel(url1, engine='openpyxl')  # Specify the engine as 'openpyxl' for reading Excel files
df2 = pd.read_csv(url2, encoding='latin1')    # Specify the encoding as 'latin1'


In [ ]:
# Merging the DataFrames
merged_df = pd.merge(df1, df2, how='inner')

The two file are merged together to form a single DataFrame called 'merged_df'

In [ ]:
merged_df.head()

In [ ]:
# Displaying Data Type
merged_df.info()

In [ ]:
# Identifying missing values
missing_data = merged_df.isnull().sum()
print(missing_data)

In [ ]:
# Calculate the mode of the 'Cuisines' column
cuisine_mode = merged_df['Cuisines'].mode().iloc[0]

# Fill missing values in the 'Cuisines' column with the mode
merged_df['Cuisines'].fillna(cuisine_mode, inplace=True)

In [ ]:
# Converting categorical columns to object type for compatibility
for col in merged_df.columns:
    if merged_df[col].dtype == 'category':
        merged_df[col] = merged_df[col].astype('object')

In [ ]:
# Extracting and describing statistics for categorical columns (both object and category types) in the DataFrame
cate_df = merged_df.select_dtypes(include=['object', 'category'])
cate_df.describe()

In [ ]:
# Selecting columns with numeric dtypes (integers and floats)
numeric_df = merged_df.select_dtypes(include=['int64', 'float64'])
numeric_df.describe()

In [ ]:
# Ploting descritive graph
plt.figure(figsize=(15, 8))
sns.heatmap(numeric_df.describe(), annot=True, fmt=".2f", linewidths=0.2, linecolor='black',cmap='coolwarm')
plt.title("Numerical Data Descriptive Statistics")
plt.show()

The analysis optimizes our DataFrame by converting object type columns to category type to reduce memory usage, and category type columns to object type for compatibility with certain operations. Extracting and describing statistics for categorical columns helps us understand the distribution and frequency of categorical data. Similarly, generating summary statistics for numeric columns provides insights into their central tendencies and variability. 

These steps ensure efficient memory usage and compatibility, enhancing the performance and accuracy of our data analysis. Ultimately, this process is
crucial for maintaining a manageable and insightful dataset, facilitating better decision-making and analysis.

In [ ]:
# Set the size of the figure
plt.figure(figsize=(10, 8))

# Create a histogram plot of the Average Cost for two column from the DataFrame merged_df
sns.histplot(data = merged_df, x = 'Aggregate rating')
plt.title('Aggregate Rating (Fig 1)')
plt.show()

In [ ]:
# Set the size of the figure
plt.figure(figsize=(9, 7))

# Create a count plot of the 'Country' column from the DataFrame merged_df
sns.countplot(data=merged_df, x='Country')

# Set plot title and axis labels
plt.title('Distribution of Restaurants by Country')
plt.xlabel('Country')
plt.ylabel('Count')

# Rotate x-axis labels for better readability if necessary
plt.xticks(rotation=45)

# Show the plot
plt.show()

In [ ]:

# Set the size of the figure
plt.figure(figsize=(12, 8))
# Create a histogram plot of the Price range for two column from the DataFrame merged_df
sns.histplot(data = merged_df, x = 'Price range')
plt.title('Price range (Fig 2)')
plt.show()

In [ ]:
import squarify

# Set the size of the figure
plt.figure(figsize=(78, 45))

# Create a treemap plot with Price range as the sizes and Cuisines as the labels from the DataFrame merged_df
squarify.plot(sizes=merged_df['Price range'], label=merged_df['Cuisines'], alpha=.8, pad=True)



# Add title and remove axes
plt.title('Cuisine Popularity Treemap', fontsize=20)
plt.axis('off')

# Create a legend
# Generate a list of unique labels and their corresponding colors
cuisines = merged_df['Cuisines'].unique()
handles = [plt.Rectangle((0, 0), 1, 1, color=plt.cm.viridis(i / float(len(cuisines)))) for i in range(len(cuisines))]

# Add the legend
plt.legend(handles, cuisines, title="Cuisines", bbox_to_anchor=(1, 1), loc='upper left')

# Display the plot
plt.show()

'''import squarify
# Set the size of the figure
plt.figure(figsize=(68,45))
# Create a treemap plot of the Price range for two column from the DataFrame merged_df

squarify.plot(sizes=merged_df['Price range'], label=merged_df['Cuisines'], alpha=.8)
plt.title('Cuisine Popularity Treemap')
plt.axis('off')
plt.show()'''

Please note that the plot above depicts a treemap of approximately 1825 varieties of cuisines.

In [ ]:
# Set the size of the figure
plt.figure(figsize=(45, 15))

# Create a count plot of the Price range for different Cuisines
sns.countplot(data=merged_df, x='Cuisines')

# Set the title and axis labels
plt.title('Count of Different Cuisines', fontsize=16)
plt.xlabel('Cuisines', fontsize=6)
plt.ylabel('Count', fontsize= 6)

# Rotate x-ticks for better readability if there are many cuisines
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.show()

The analysis creates histograms to visualize the Aggregate rating and Price range distribution, helping us understand their spread and central tendencies. 

A treemap illustrates the relative popularity of different cuisines by plotting the Price range against Cuisines, revealing which cuisines are most 
prevalent. A count plot shows the frequency of different cuisines, aiding in identifying the most common ones. These steps are crucial for effective data visualization and optimization, providing clearer insights and more efficient data handling for better decision-making.

In [ ]:
# Correlation Analysis for numeric_df is the DataFrame
correlation_matrix = numeric_df.corr()

# Plot the correlation matrix including the target variable
plt.figure(figsize=(15, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

The correlation coefficients offer valuable insights into the dataset's dynamics. With a coefficient of 0.81 between average price and vote, a strong 
positive relationship emerges, indicating that higher restaurant prices often correlate with higher votes, potentially reflecting superior quality or 
satisfaction. Similarly, a coefficient of 0.51 between vote and price range suggests a moderate positive connection, implying that establishments with 
broader price ranges tend to garner more votes. Additionally, the correlation of 0.47 between average cost for two and price range underscores how 
pricing influences customer engagement, suggesting that venues with wider price ranges attract a more diverse clientele.

On the other hand, the negative correlation of -0.33 between aggregate rating and restaurant ID suggests that as the identification number rises, 
there's a tendency for aggregate ratings to decrease slightly, hinting at variations in rating patterns across different restaurant IDs. Essentially, 
this implies that newer establishments, indicated by higher ID numbers, may receive marginally lower aggregate ratings compared to their older 
counterparts. These findings highlight the substantial impact of pricing and identification variables on customer engagement and perception within 
the dataset, underscoring their pivotal role in subsequent analyses and decision-making processes.

In [ ]:
# Identifying Outliers

# Calculate z-scores for each data point
z_scores = ((numeric_df - numeric_df.mean()) / numeric_df.std()).abs()

# Define threshold for outliers (if z-score greater than 3)
outlier_threshold = 3

# Identify outliers using the z-score method
outlier_conditions = (z_scores > outlier_threshold)

# Display rows containing outliers
outliers = merged_df[outlier_conditions.any(axis=1)]
print("Rows with outliers:")
print(outliers)

# Visualize the outliers using a boxplot
plt.figure(figsize=(9, 7))  
sns.boxplot(data= numeric_df) 
plt.title("Boxplot with Outliers")
plt.show()


In [ ]:
# Handling outliers
# Step 1: Identify Variables with Outliers
outlier_vars = ['Average Cost for two', 'Price range', 'Aggregate rating','Votes']

# Step 2: Apply Logarithmic Transformation
# Logarithmic Transformation
log_transformed_vars = merged_df[outlier_vars].apply(lambda x: np.log(x + 1))  # Adding 1 to avoid log(0) issues

# Replace the original columns with the transformed columns
merged_df[outlier_vars] = log_transformed_vars  

# Visualize the transformed data
plt.figure(figsize=(10, 8))
for i, col in enumerate(outlier_vars):
    plt.subplot(3, 3, i+1)
    plt.hist(log_transformed_vars[col], bins=30, alpha=0.5, color='blue', label='Log Transformation')
    plt.legend()
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
# Show missing value
merged_df.isnull().sum()

In [ ]:
# PCA Analysis
from sklearn.decomposition import PCA

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

# Perform PCA
pca = PCA()
pca.fit(scaled_data)

# Transform the data into the principal components
pca_data = pca.transform(scaled_data)

# Extracting explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# df_columns is a list of column names in the df1 DataFrame
df_columns = numeric_df.columns.tolist()

# Create a DataFrame to store the variable loadings for each principal component
loadings_df = pd.DataFrame(pca.components_, columns=df_columns)

# Print the variable loadings for each principal component
for i in range(loadings_df.shape[0]):
    print(f"Principal Component {i+1} Loadings:")
    print(loadings_df.iloc[i].sort_values(ascending=False))
    print()
# Create a DataFrame to examine the principal components
pca_df = pd.DataFrame(data=pca_data, columns=[f"PC{i+1}" for i in range(pca_data.shape[1])])

# Visualize the explained variance ratio
print("Explained Variance Ratio:", explained_variance_ratio)


In [ ]:
# Create a DataFrame to examine the principal components
pca_df = pd.DataFrame(data=pca_data, columns=[f"PC{i+1}" for i in range(pca_data.shape[1])])

# Scree plot
plt.figure(figsize=(10, 4))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o', linestyle='--')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.xticks(range(1, len(explained_variance_ratio) + 1))
plt.show()

# Scatter plot of the first two principal components
plt.figure(figsize=(10, 4))
plt.scatter(pca_df['PC1'], pca_df['PC2'], alpha=0.5)
plt.title('PCA - PC1 vs PC2')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

The elongated clusters observed in PCA1 and PCA2 indicate a clear directional trend in the data. In contrast, the outlier
points could be indicative of data points with unique characteristics that differentiate them from the main clusters or could
potentially be errors in the data.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data["Variable"] = numeric_df.columns

# Calculate VIF for each predictor variable
vif_data["VIF"] = [variance_inflation_factor(numeric_df.values, i) for i in range(len(numeric_df.columns))]

# Print the VIF DataFrame
print(vif_data)

The variance inflation factors (VIFs) for the predictor variables are all less than 10, indicating that the data do not exhibit extreme multicollinearity.

In [ ]:
# Defining features
features = ['Country Code', 'Restaurant ID', 'Longitude', 'Latitude', 'Price range', 'Aggregate rating', 'Votes']
categorical_features = ['Country', 'Restaurant Name', 'City', 'Address', 'Locality', 'Locality Verbose', 'Cuisines', 'Currency', 'Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu', 'Rating color', 'Rating text']																

# Split the dataset into features (X) and the target variable (y)
X_numerical = merged_df[features]
X_categorical = pd.get_dummies(merged_df[categorical_features])  # One-hot encode categorical columns
X = pd.concat([X_numerical, X_categorical], axis=1)  # Combine numerical and encoded categorical features
y = merged_df['Average Cost for two']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Fit and transform the scaled features

# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=123)

# Initialize and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Print the coefficients
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': lr_model.coef_})
coefficients_sorted = coefficients.reindex(coefficients['Coefficient'].abs().sort_values(ascending=False).index)
print(coefficients_sorted)

In [ ]:
print(lr_model.coef_, X.columns)

In [ ]:
# Set the random seed
np.random.seed(123)

# Split the dataset into features (X) and the target variable (y)
Xi = merged_df[['Country Code', 'Longitude', 'Latitude', 'Price range', 'Aggregate rating', 'Votes']]  # Features (excluding categorical columns)
X_categoricali = pd.get_dummies(merged_df[['Has Table booking', 'Rating color', 'Rating text', 'Cuisines', 'Currency', 'Locality', 'Locality Verbose']])  # One-hot encode categorical columns
Xi = pd.concat([Xi, X_categoricali], axis=1)  # Combine numeric al and encoded categorical features
yi = merged_df['Average Cost for two']

# Standardize features 
scaler = StandardScaler()
X_scaledi = scaler.fit_transform(Xi)  # Fit and transform the scaled features

# Split the dataset into training (80%) and testing (20%) sets
X_traini, X_testi, y_traini, y_testi = train_test_split(X_scaledi, yi, test_size=0.2)

In [ ]:
# Set the random seed
np.random.seed(123)

# Define regression models
regressors = {
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR()
}

# Evaluate and print results for each regressor
for name, reg in regressors.items():
    # Perform 5-fold cross-validation for MSE
    scores_mse = cross_val_score(reg, Xi, yi, cv=5, scoring='neg_mean_squared_error')
    # Perform 5-fold cross-validation for R²
    scores_r2 = cross_val_score(reg, Xi, yi, cv=5, scoring='r2')
    
    # Compute mean and standard deviation of each metric
    mean_mse, std_mse = -scores_mse.mean(), scores_mse.std()
    mean_r2, std_r2 = scores_r2.mean(), scores_r2.std()
    
    # Print the results
    print(f"Regressor: {name}")
    print(f"Mean Squared Error: {mean_mse:.4f} +/- {std_mse:.4f}")
    print(f"R-squared (R²) Score: {mean_r2:.4f} +/- {std_r2:.4f}")
    print("="*50)

RandomForest has the lowest MSE (0.9298), followed by GradientBoosting (0.9635), SVR (1.0231), and ElasticNet (1.2164). In terms of R-squared scores, RandomForest again leads with 0.5378, followed by GradientBoosting at 0.5151, SVR at 0.2543, and ElasticNet at -0.0304. A negative R-squared score, as seen with ElasticNet, indicates that the model performs worse than simply predicting the average of the dependent variable. Given these results, hyperparameter tuning will be conducted using RandomForest to further improve the model.

In [ ]:
# Set the random seed
np.random.seed(123)

# Define the Random Forest regressor
random_forest = RandomForestRegressor()

# Evaluate the Random Forest regressor using 5-fold cross-validation for MSE and R²
scores_mse = cross_val_score(random_forest, Xi, yi, cv=5, scoring='neg_mean_squared_error')
scores_r2 = cross_val_score(random_forest, Xi, yi, cv=5, scoring='r2')

# Compute mean and standard deviation of each metric
mean_mse, std_mse = -scores_mse.mean(), scores_mse.std()
mean_r2, std_r2 = scores_r2.mean(), scores_r2.std()

# Print the results
print("Regressor: Random Forest")
print(f"Mean Squared Error: {mean_mse:.4f} +/- {std_mse:.4f}")
print(f"R-squared (R²) Score: {mean_r2:.4f} +/- {std_r2:.4f}")
print("="*50)

# Train the Random Forest regressor on the full training set
random_forest.fit(X_traini, y_traini)

# Save the trained model using joblib
joblib_filename = "RandomForest_model.joblib"
joblib.dump(random_forest, joblib_filename)
print(f"Saved Random Forest model to {joblib_filename}")

print('Hypertuning')

'''
# Set the random seed for reproducibility
np.random.seed(123)

# Initialize the RandomForestRegressor with a random state for reproducibility
random_forest = RandomForestRegressor(random_state=42)

# Define the hyperparameter space using distributions for RandomizedSearchCV
param_dist = {
    "n_estimators": randint(100, 1000),  # Number of trees (100 to 1000)
    "max_depth": randint(3, 10),  # Maximum depth of trees (3 to 9)
    "min_samples_split": randint(2, 10),  # Minimum samples to split a node (2 to 9)
    "min_samples_leaf": randint(1, 5)  # Minimum samples allowed in a leaf node (1 to 4)
}

# Perform RandomizedSearchCV for RandomForestRegressor
random_search_rf = RandomizedSearchCV(estimator=random_forest, param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2, random_state=42)
random_search_rf.fit(X_traini, y_traini)

# Print the best parameters found for RandomForest
print("Best parameters for RandomForestRegressor found:")
print(random_search_rf.best_params_)

# Get the best model from RandomizedSearchCV
best_rf_model = random_search_rf.best_estimator_

# Evaluate the best model on the test set
def evaluate_model(model, X_testi, y_testi):
    y_pred = model.predict(X_testi)
    mse = mean_squared_error(y_testi, y_pred)
    r2 = r2_score(y_testi, y_pred)
    mae = mean_absolute_error(y_testi, y_pred)
    return mse, r2, mae

# Evaluate RandomForestRegressor
mse_rf, r2_rf, mae_rf = evaluate_model(best_rf_model, X_testi, y_testi)
print(f"RandomForestRegressor Test Set Performance:")
print(f"Mean Squared Error: {mse_rf:.4f}")
print(f"R-squared (R²) Score: {r2_rf:.4f}")
print(f"Mean Absolute Error (MAE): {mae_rf:.4f}")
print("="*50)

# Set the random seed for reproducibility
np.random.seed(123)

# Initialize the RandomForestRegressor with a random state for reproducibility
random_forest = RandomForestRegressor(random_state=42)

# Define the hyperparameter space
param_dist = {
    "n_estimators": range(100, 1000, 100),  # Number of trees (100 to 1000 with steps of 100)
    "max_depth": range(3, 10),  # Maximum depth of trees (3 to 9)
    "min_samples_split": range(2, 10),  # Minimum samples to split a node (2 to 9)
    "min_samples_leaf": range(1, 5)  # Minimum samples allowed in a leaf node (1 to 4)
}

# Perform GridSearchCV for RandomForestRegressor
grid_search_rf = GridSearchCV(estimator=random_forest, param_grid=param_dist, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search_rf.fit(X_traini, y_traini)

# Print the best parameters found for RandomForest
print("Best parameters for RandomForestRegressor found:")
print(grid_search_rf.best_params_)

# Get the best model from GridSearchCV
best_rf_model = grid_search_rf.best_estimator_

# Evaluate the best model on the test set
def evaluate_model(model, X_testi, y_testi):
    y_pred = model.predict(X_testi)
    mse = mean_squared_error(y_testi, y_pred)
    r2 = r2_score(y_testi, y_pred)
    mae = mean_absolute_error(y_testi, y_pred)
    return mse, r2, mae

# Evaluate RandomForestRegressor
mse_rf, r2_rf, mae_rf = evaluate_model(best_rf_model, X_testi, y_testi)
print(f"RandomForestRegressor Test Set Performance:")
print(f"Mean Squared Error: {mse_rf:.4f}")
print(f"R-squared (R²) Score: {r2_rf:.4f}")
print(f"Mean Absolute Error (MAE): {mae_rf:.4f}")
print("="*50)'''

While the Random Forest regressor shows promise with an average R² of 0.4586, the high standard deviations indicate that the 
model's performance is not stable. Addressing the variability and refining the model could lead to improved results. However, 
the computational requirements for such improvements are too demanding for my machine.

In [ ]:
# Set the random seed
np.random.seed(123)

# Split the dataset into features (X) and the target variable (y)
Xii = merged_df[['Country Code', 'Longitude', 'Latitude', 'Average Cost for two', 'Aggregate rating', 'Votes']]  # Features (excluding categorical columns)
X_categoricalii = pd.get_dummies(merged_df[['Has Table booking', 'Rating color', 'Rating text', 'Cuisines', 'Currency', 'Locality', 'Locality Verbose']])  # One-hot encode categorical columns
Xii = pd.concat([Xii, X_categoricalii], axis=1)  # Combine numerical and encoded categorical features
yii = merged_df['Price range']

# Standardize features 
scaler = StandardScaler()
X_scaledii = scaler.fit_transform(Xii)  # Fit and transform the scaled features

# Split the dataset into training (80%) and testing (20%) sets
X_trainii, X_testii, y_trainii, y_testii = train_test_split(X_scaledii, yii, test_size=0.2)

In [ ]:
# Set the random seed
np.random.seed(123)

# Define regression models
regressors = {
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR()
}

# Evaluate and print results for each regressor
for name, reg in regressors.items():
    # Perform 5-fold cross-validation for MSE
    scores_mse = cross_val_score(reg, Xii, yii, cv=5, scoring='neg_mean_squared_error')
    # Perform 5-fold cross-validation for R²
    scores_r2 = cross_val_score(reg, Xii, yii, cv=5, scoring='r2')
    
    # Compute mean and standard deviation of each metric
    mean_mse, std_mse = -scores_mse.mean(), scores_mse.std()
    mean_r2, std_r2 = scores_r2.mean(), scores_r2.std()
    
    # Print the results
    print(f"Regressor: {name}")
    print(f"Mean Squared Error: {mean_mse:.4f} +/- {std_mse:.4f}")
    print(f"R-squared (R²) Score: {mean_r2:.4f} +/- {std_r2:.4f}")
    print("="*50)
    

Given these insights, while both GradientBoosting and RandomForest are strong contenders, GradientBoosting slightly edges out as
the top performer.

In [ ]:
print('Hypertuning')

''' from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Set the random seed for reproducibility
np.random.seed(42)

# Initialize the GradientBoostingRegressor with a random state for reproducibility
gradient_boosting = GradientBoostingRegressor(random_state=42)

# Define the hyperparameter space
param_dist = {
    "n_estimators": range(100, 1000, 100),  # Number of boosting stages (100 to 1000 with steps of 100)
    "learning_rate": [0.1, 0.01, 0.05],  # Learning rates to try
    "max_depth": range(3, 8),  # Maximum depth of trees (3 to 7)
    "min_samples_split": range(2, 10),  # Minimum samples to split a node (2 to 9)
    "min_samples_leaf": range(1, 5),  # Minimum samples in a leaf node (1 to 4)
    "loss": ["squared_error", "huber"]  # Different loss functions
}

# Perform GridSearchCV for GradientBoostingRegressor
grid_search_gb = GridSearchCV(estimator=gradient_boosting, param_grid=param_dist, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search_gb.fit(X_trainii, y_trainii)

# Print the best parameters found for GradientBoostingRegressor
print("Best parameters for GradientBoostingRegressor found:")
print(grid_search_gb.best_params_)

# Get the best model from GridSearchCV
best_gb_model = grid_search_gb.best_estimator_

# Evaluate the best model on the test set
def evaluate_model(model, X_testii, y_testii):
    y_pred = model.predict(X_testii)
    mse = mean_squared_error(y_testii, y_pred)
    r2 = r2_score(y_testii, y_pred)
    mae = mean_absolute_error(y_testii, y_pred)
    return mse, r2, mae

# Evaluate GradientBoostingRegressor
mse_gb, r2_gb, mae_gb = evaluate_model(best_gb_model, X_testii, y_testii)
print(f"GradientBoostingRegressor Test Set Performance:")
print(f"Mean Squared Error: {mse_gb:.4f}")
print(f"R-squared (R²) Score: {r2_gb:.4f}")
print(f"Mean Absolute Error (MAE): {mae_gb:.4f}")
print("="*50)'''

In [ ]:
# Set the random seed
np.random.seed(123)

# Define regression models
regressor = GradientBoostingRegressor()


# Perform 5-fold cross-validation for MSE
scores_mse = cross_val_score(regressor, Xii, yii, cv=5, scoring='neg_mean_squared_error')
# Perform 5-fold cross-validation for R²
scores_r2 = cross_val_score(regressor, Xii, yii, cv=5, scoring='r2')

# Compute mean and standard deviation of each metric
mean_mse, std_mse = -scores_mse.mean(), scores_mse.std()
mean_r2, std_r2 = scores_r2.mean(), scores_r2.std()

# Print the results
print("Regressor: Gradient Boosting")
print(f"Mean Squared Error: {mean_mse:.4f} +/- {std_mse:.4f}")
print(f"R-squared (R²) Score: {mean_r2:.4f} +/- {std_r2:.4f}")
print("="*50)

# Train the Random Forest regressor on the full training set
random_forest = RandomForestRegressor()
random_forest.fit(X_traini, y_traini)

# Save the trained Random Forest model using joblib
joblib_filename = "RandomForest_modelZomato.joblib"
joblib.dump(random_forest, joblib_filename)
print(f"Saved Random Forest model to {joblib_filename}")

The Gradient Boosting regressor shows strong performance with an average R² of 0.7280 and a low MSE of 0.0066. However, the
high standard deviations for both metrics indicate significant variability in model performance across different data subsets.
Improving data preprocessing, hyperparameter tuning, could enhance model stability and performance. Given the computational
demands simpler models might be necessary.

Zomato Data Analysis is invaluable for foodies seeking the best cuisines worldwide that fit within their budget. The analysis 
helps individuals find value-for-money restaurants in different regions for various cuisines. This analysis also aids those 
looking to experience the finest local cuisines of a country and identifies the localities with the highest concentration of 
restaurants serving those cuisines. While the Random Forest regressor showed promise, the high variability in its performance 
indicated instability. However, the Gradient Boosting regressor demonstrated strong and more consistent performance, making it
a better choice for this analysis despite potential computational demands.